# Make Dataset

## Imports

In [1]:
import sys
sys.path.append("../src")

In [2]:
from data.wav_data_reader import WavDataReader
from data.signal_dataset import SignalDataset
from features.wav_feature_extractor import WavFeatureExtractor
from features.features_dataset import FeaturesDataset
import data.preparation_eurythmy_data as ped

In [8]:
signal_dataset_path= r"..\data\processed\raw_signal_dataset.pkl"
signal_dataset= SignalDataset.load(signal_dataset_path)
signal_dataset.features.shape

(625, 6)

In [4]:

letter_dictionary= ped.return_letter_dictionary(signal_dataset.features['id_measurement'].tolist())
print(letter_dictionary)


{1: {'A1': [17, 26], 'G1': [27, 36], 'D1': [37, 46], 'A2': [47, 56], 'G2': [57, 66], 'D2': [67, 77], 'A3': [79, 88], 'G3': [89, 98], 'D3': [99, 110], 'A4': [112.0, 120.0], 'G4': [121.0, 131.0], 'D4': [132.0, 143.0]}, 2: {'A1': [17, 26], 'G1': [27, 36], 'D1': [37, 46], 'A2': [47, 56], 'G2': [57, 66], 'D2': [67, 77], 'A3': [79, 88], 'G3': [89, 98], 'D3': [99, 110], 'A4': [112.0, 120.0], 'G4': [121.0, 131.0], 'D4': [132.0, 143.0]}, 3: {'A1': [17, 26], 'G1': [27, 36], 'D1': [37, 46], 'A2': [47, 56], 'G2': [57, 66], 'D2': [67, 77], 'A3': [79, 88], 'G3': [89, 98], 'D3': [99, 110], 'A4': [112.0, 120.0], 'G4': [121.0, 131.0], 'D4': [132.0, 143.0]}, 4: {'A1': [17, 26], 'G1': [27, 36], 'D1': [37, 46], 'A2': [47, 56], 'G2': [57, 66], 'D2': [67, 77], 'A3': [79, 88], 'G3': [89, 98], 'D3': [99, 110], 'A4': [112.0, 120.0], 'G4': [121.0, 131.0], 'D4': [132.0, 143.0]}, 5: {'A1': [17, 26], 'G1': [27, 36], 'D1': [37, 46], 'A2': [47, 56], 'G2': [57, 66], 'D2': [67, 77], 'A3': [79, 88], 'G3': [89, 98], 'D3

In [5]:
signal_dataset.features.head(100)

,id_measurement,id_performance,datetime,plant,generation,num_eurythmy
0,1,1,2023-04-29,salad,1,1
1,2,1,2023-04-29,salad,1,1
2,3,1,2023-04-29,salad,1,1
3,4,1,2023-04-29,salad,2,0
4,5,1,2023-04-29,salad,2,0
...,...,...,...,...,...,...
95,96,16,2023-06-01,tomato,1,0
96,97,17,2023-06-06,salad,1,1
97,98,17,2023-06-06,salad,1,1
98,99,17,2023-06-06,salad,1,1


In [6]:
signal_dataset.trim_signals_and_update_features('id_measurement',letter_dictionary)
signal_dataset.features.head(10)

,id_measurement,id_performance,datetime,plant,generation,num_eurythmy,segment
0,1,1,2023-04-29,salad,1,1,None
1,2,1,2023-04-29,salad,1,1,None
2,3,1,2023-04-29,salad,1,1,None
3,4,1,2023-04-29,salad,2,0,None
4,5,1,2023-04-29,salad,2,0,None
5,6,1,2023-04-29,salad,2,0,None
6,7,2,2023-04-29,salad,1,1,None
7,8,2,2023-04-29,salad,1,1,None
8,9,2,2023-04-29,salad,1,1,None
9,10,2,2023-04-29,salad,1,0,None


In [7]:
klk

NameError: name 'klk' is not defined

## Reader

In [ ]:
#wav_folder= r"..\data\raw\wav_files"
wav_folder= r"..\data\interim\testing"

In [ ]:
# Initialize the reader with the folder of wavs
reader= WavDataReader(folder= wav_folder)

# Get the signals and the keys from the reader
signals, ids= reader.get_values_and_keys()

# Get the measurement labels from the keys of the files
meas_df= ped.return_meas_labels_by_keys(ids)

In [ ]:
meas_df.head()

In [ ]:
klk

## Signal Dataset

In [ ]:
%%time
# Initialize SignalDataset with the signals and features/labels
signal_dataset= SignalDataset(signals= signals, features= meas_df)

# Standardize Signals using Zscore
#signal_dataset.standardize_signals("zscore")

# Segment Signals in 1sec segments
#signal_dataset.segment_signals(segment_duration=1)

# Remove signals whose values are all equal
#signal_dataset.remove_constant_signals()

#Add eurythmy letter data to the features
#df= ped.add_meas_letters(signal_dataset.features)

In [ ]:
signal_dataset.features.shape

In [ ]:
signal_dataset.features.head()

In [ ]:
signal_dataset_path= r"..\data\processed\raw_signal_dataset.pkl"
#signal_dataset.save(signal_dataset_path)

In [ ]:
klk

## Feature Extractor

In [ ]:
extractor= WavFeatureExtractor(sample_rate= 10000, mfccs= True, temporal= True, statistical= True)

## Features Dataset

In [ ]:
%%time
feat_dataset= FeaturesDataset(signal_dataset,extractor)
feat_dataset.features.shape

In [ ]:
feat_dataset.features.head()

In [ ]:
# Save Dataset
feat_dataset_path= r"..\data\processed\features_letter_dataset.pkl"
feat_dataset.save(feat_dataset_path)
feat_dataset_path_csv= r"..\data\processed\features_letter_dataset.csv"
feat_dataset.save_to_csv(feat_dataset_path_csv)